# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data
 

{'companyName': 'Apple Inc',
 'marketcap': 2263543238271,
 'week52high': 145.68,
 'week52low': 89.2,
 'week52highSplitAdjustOnly': 149.27,
 'week52lowSplitAdjustOnly': 89.09,
 'week52change': 0.5343633780597634,
 'sharesOutstanding': 17325523817,
 'float': 0,
 'avg10Volume': 71014285,
 'avg30Volume': 73330717,
 'day200MovingAvg': 132.32,
 'day50MovingAvg': 131.44,
 'employees': 151528,
 'ttmEPS': 4.52,
 'ttmDividendRate': 0.8583641354038541,
 'dividendYield': 0.006445409690217895,
 'nextDividendDate': '',
 'exDividendDate': '2021-05-05',
 'nextEarningsDate': '2021-07-29',
 'peRatio': 29.643615050975228,
 'beta': 1.6130629757473371,
 'maxChangePercent': 50.02559686830834,
 'year5ChangePercent': 4.802255897374938,
 'year2ChangePercent': 1.835652362183389,
 'year1ChangePercent': 0.5560425874395171,
 'ytdChangePercent': -0.013945076229897525,
 'month6ChangePercent': 0.07727681776294291,
 'month3ChangePercent': 0.08025783231278348,
 'month1ChangePercent': 0.02413139256777357,
 'day30ChangeP

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.5560425874395171

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_url_call = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url_call).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index = my_columns
            ), ignore_index = True
        )

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,67.27,3.246263,N/A
1,179,FCX,40.55,2.848504,N/A
2,208,GPS,33.29,2.17776,N/A
3,441,TPR,43.27,2.032872,N/A
4,253,IVZ,29.33,1.953422,N/A
5,410,SIVB,565.63,1.666417,N/A
6,344,NUE,104.60,1.622653,N/A
7,314,MOS,33.76,1.573573,N/A
8,149,DXC,42.14,1.547945,N/A
9,385,PWR,95.64,1.540505,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size 
    portfolio_size = input('Enter the size of your portfolio: ')
    try:
        float(portfolio_size)
    except ValueError:
        print('Enter a number.')
        portfolio_size = input('Enter the size of your portfolio: ')
portfolio_input()

Enter the size of your portfolio: 50000


In [9]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
for symbol_string in symbol_strings:
    batch_api_url_call = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url_call).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                     data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                     data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index = hqm_columns 
            ), 
            ignore_index = True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,147.93,N/A,0.709232,N/A,0.242485,N/A,0.199041,N/A,0.102673,N/A,N/A
1,AAL,24.09,N/A,0.384446,N/A,0.394274,N/A,-0.01631,N/A,0.027191,N/A,N/A
2,AAP,203.61,N/A,0.489081,N/A,0.269185,N/A,0.111807,N/A,-0.021551,N/A,N/A
3,AAPL,136.61,N/A,0.557548,N/A,0.077981,N/A,0.083157,N/A,0.02481,N/A,N/A
4,ABBV,120.50,N/A,0.319108,N/A,0.136855,N/A,0.082526,N/A,-0.009202,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.57,N/A,0.329782,N/A,0.129669,N/A,0.132006,N/A,-0.006102,N/A,N/A
501,ZBH,161.73,N/A,0.276205,N/A,0.108391,N/A,-0.033978,N/A,-0.06084,N/A,N/A
502,ZBRA,524.25,N/A,0.984357,N/A,0.342146,N/A,0.081946,N/A,0.049349,N/A,N/A
503,ZION,54.23,N/A,0.625265,N/A,0.357495,N/A,-0.080803,N/A,-0.107146,N/A,N/A


# Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]
hqm_dataframe.replace([None], 0, inplace = True)

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,147.93,N/A,0.709232,0.754455,0.242485,0.560396,0.199041,0.885149,0.102673,0.912871,N/A
1,AAL,24.09,N/A,0.384446,0.419802,0.394274,0.837624,-0.016310,0.164356,0.027191,0.649505,N/A
2,AAP,203.61,N/A,0.489081,0.538614,0.269185,0.627723,0.111807,0.633663,-0.021551,0.312871,N/A
3,AAPL,136.61,N/A,0.557548,0.613861,0.077981,0.2,0.083157,0.522772,0.024810,0.631683,N/A
4,ABBV,120.50,N/A,0.319108,0.314851,0.136855,0.328713,0.082526,0.518812,-0.009202,0.394059,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.57,N/A,0.329782,0.330693,0.129669,0.306931,0.132006,0.720792,-0.006102,0.427723,N/A
501,ZBH,161.73,N/A,0.276205,0.255446,0.108391,0.259406,-0.033978,0.126733,-0.060840,0.130693,N/A
502,ZBRA,524.25,N/A,0.984357,0.89703,0.342146,0.762376,0.081946,0.514851,0.049349,0.770297,N/A
503,ZION,54.23,N/A,0.625265,0.691089,0.357495,0.790099,-0.080803,0.065347,-0.107146,0.019802,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop = True)
#hqm_dataframe.drop('index', axis = 'columns', inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DXC,42.54,N/A,1.534068,0.982178,0.769930,0.986139,0.430530,0.994059,0.118635,0.940594,0.975743
1,DVN,29.23,N/A,1.409123,0.974257,0.925733,0.99802,0.206597,0.89703,0.106197,0.924752,0.948515
2,AMAT,143.34,N/A,1.492262,0.978218,0.590843,0.960396,0.218715,0.90495,0.120766,0.948515,0.94802
3,NVDA,732.76,N/A,1.042416,0.910891,0.369159,0.80198,0.412293,0.992079,0.270232,0.99802,0.925743
4,FTNT,242.45,N/A,0.789556,0.807921,0.777642,0.988119,0.229797,0.926733,0.126035,0.956436,0.919802
5,IRM,47.68,N/A,0.804616,0.817822,0.628101,0.968317,0.275797,0.972277,0.097904,0.90099,0.914851
6,TGT,240.25,N/A,1.044350,0.914851,0.377073,0.815842,0.304383,0.978218,0.109410,0.928713,0.909406
7,F,14.96,N/A,1.366130,0.970297,0.677650,0.978218,0.113306,0.637624,0.256441,0.99604,0.895545
8,EXR,167.11,N/A,0.738280,0.776238,0.493613,0.926733,0.228798,0.922772,0.115522,0.936634,0.890594
9,HES,89.31,N/A,0.857719,0.849505,0.660177,0.972277,0.231682,0.930693,0.056321,0.8,0.888119


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the size of your portfolio: 50000


In [15]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

C:\Users\hp\Desktop\Projects\algorithmic-trading-python\env\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DXC,42.54,23,1.534068,0.982178,0.769930,0.986139,0.430530,0.994059,0.118635,0.940594,0.975743
1,DVN,29.23,34,1.409123,0.974257,0.925733,0.99802,0.206597,0.89703,0.106197,0.924752,0.948515
2,AMAT,143.34,6,1.492262,0.978218,0.590843,0.960396,0.218715,0.90495,0.120766,0.948515,0.94802
3,NVDA,732.76,1,1.042416,0.910891,0.369159,0.80198,0.412293,0.992079,0.270232,0.99802,0.925743
4,FTNT,242.45,4,0.789556,0.807921,0.777642,0.988119,0.229797,0.926733,0.126035,0.956436,0.919802
5,IRM,47.68,20,0.804616,0.817822,0.628101,0.968317,0.275797,0.972277,0.097904,0.90099,0.914851
6,TGT,240.25,4,1.044350,0.914851,0.377073,0.815842,0.304383,0.978218,0.109410,0.928713,0.909406
7,F,14.96,66,1.366130,0.970297,0.677650,0.978218,0.113306,0.637624,0.256441,0.99604,0.895545
8,EXR,167.11,5,0.738280,0.776238,0.493613,0.926733,0.228798,0.922772,0.115522,0.936634,0.890594
9,HES,89.31,11,0.857719,0.849505,0.660177,0.972277,0.231682,0.930693,0.056321,0.8,0.888119


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
columns_formats = {
    'A':['Ticker', string_template],
    'B':['Price', dollar_template],
    'C':['Number of Shares to Buy', integer_template],
    'D':['One-Year Price Return', percent_template],
    'E':['One-Year Return Percentile', percent_template],
    'F':['Six-Month Price Return', percent_template],
    'G':['Six-Month Return Percentile', percent_template],
    'H':['Three-Month Price Return', percent_template],
    'I':['Three-Month Return Percentile', percent_template],
    'J':['One-Month Price Return', percent_template],
    'K':['One-Month Return Percentile', percent_template],
    'L':['HQM Score', percent_template]
}
for column in columns_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 22, columns_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', columns_formats[column][0], columns_formats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()

C:\Users\hp\Desktop\Projects\algorithmic-trading-python\env\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
